# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Assignment 6: ETL pipeline for Text Mining and Analytics

At the end of the experiment, you will be able to:

* perform text mining and analytics using Spark SQL functions
* use Spark’s built-in and external data sources to write data in different file formats as part of the extract, transform, and load (ETL) tasks


## Information

The basic terminology related to text analytics are

* **Text**: a sequence of words and punctuation
* **Corpus**: a large body of text
* **Frequency distribution**: the frequency of words in a text object
* **Collocation**: a sequence of words that occur together unusually often
* **Bigrams**: word pairs. High frequent bigrams are collocations
* **Text normalization**: the process of transforming text into a single canonical form, e.g., converting text to lowercase, removing punctuations and stop words.

### Introduction

Text analytics is the process of deriving information from text. It usually involves information retrieval, lexical analysis to study word frequency distributions, pattern recognition, tagging, information extraction, visualization, and predictive analytics. The overarching goal is, essentially, to turn text into data for analysis, via application of natural language processing (NLP), different types of algorithms and analytical methods.

Here we will consider `milton-paradise.txt` text file from Gutenberg corpus to do text mining and analytics. Starting from data extraction, we will perform various transformations on text including tokenization, the number of words counting, POS tagging, chunking and then store it in different file formats.

### Install Pyspark

In [ ]:
!pip install pyspark

### Import required packages

In [ ]:
from pyspark.sql import SparkSession
from matplotlib import pyplot as plt
import pandas as pd
import string
from pyspark.ml.feature import NGram
from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql.functions import *
import nltk

### Start a Spark Session

Spark session is a combined entry point of a Spark application, which came into implementation from Spark 2.0. Instead of having various context, everything is now encapsulated in a Spark session.

In [ ]:
# Start spark session
spark = SparkSession.builder.appName('ETL text data').getOrCreate()
spark

### Text Analytics

#### Get the text data

The raw text is from the Gutenberg corpus from the nltk package. Get file ids in Gutenberg corpus.

In [ ]:
nltk.download('gutenberg')

# Download dependencies for sent_tokenize()
nltk.download('punkt')

In [ ]:
from nltk.corpus import gutenberg
gutenberg_fileids = gutenberg.fileids()
gutenberg_fileids

The file id is `milton-paradise.txt`. Use the nltk.sent_tokenize() function to split text into sentences.

In [ ]:
milton_paradise = gutenberg.raw('milton-paradise.txt')

pdf = pd.DataFrame({'sentences': nltk.sent_tokenize(milton_paradise)})
d = spark.createDataFrame(pdf)
d.show(1, truncate= False)

From above it can be seen that empty spaces are present in the data.

#### Transform Data

* Remove trailing spaces

In [ ]:
# Transform data
d1 = d.withColumn("sentences", regexp_replace(col("sentences"), "\\s+","_"))       # replace all spaces with underscore
d1 = d1.withColumn("sentences", regexp_replace(col("sentences"), "_"," "))         # replace all underscores with one space
d1 = d1.withColumn("sentences", trim(col("sentences")))                            # remove trailing spaces

In [ ]:
d1.show(5, truncate= False)

In [ ]:
# Check for empty lines
d1.where(col("sentences")=="").count()

##### Word Tokenization

It is the process of breaking down a paragraph, a sentence or a complete text corpus into an array of words.

In [ ]:
from nltk.tokenize import word_tokenize

word_udf = udf(lambda x: word_tokenize(x), ArrayType(StringType()))
d2 = d1.withColumn("words", word_udf("sentences"))

In [ ]:
d2.show(5)

From above it can be seen that data has punctuations in it.

* **Remove punctuation and stopwords**

In [ ]:
# Download stopwords
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:

punctuation = string.punctuation
print(punctuation)

In [ ]:
# Transform data
punct_udf = udf(lambda x: [w for w in x if not w in punctuation if not w in stop_words])
d3 = d2.withColumn("words", punct_udf("words"))
d3.show(5)

In [ ]:
# Convert dataframe column to arraytype for further processing

array_udf = udf(lambda x: x, ArrayType(StringType()))
d4 = d3.withColumn("words", array_udf("words"))
d4.show(5)

##### Ngrams and collocations

Collocation is a sequence of words that occur together unusually often.
Bigrams: word pairs. High frequent bigrams are collocations.

Let's see how we transform texts to 2-grams, 3-grams, and 4-grams collocations.

In [ ]:
ngrams = [NGram(n=n, inputCol='words', outputCol=str(n)+'-grams') for n in [2,3,4]]

# build pipeline model
pipeline = Pipeline(stages=ngrams)

# transform data
texts_ngrams = pipeline.fit(d4).transform(d4)

In [ ]:
# display result
texts_ngrams.select('2-grams').show(6, truncate=False)
texts_ngrams.select('3-grams').show(6, truncate=False)
texts_ngrams.select('4-grams').show(6, truncate=False)

* Add the number of words column

In [ ]:
# Transform data
len_udf = udf(lambda x: len(x), IntegerType())

d5 = d4.withColumn("no_of_words", len_udf("words"))

In [ ]:
d5.show(5)

##### **POS (part-of-speech) tagging**

It is a process of converting a sentence to forms – list of words, list of tuples (where each tuple is having a form (word, tag)). The tag is a part-of-speech tag and signifies whether the word is a noun, adjective, verb, and so on.

To know more about POS tagging click [here](https://medium.com/greyatom/learning-pos-tagging-chunking-in-nlp-85f7f811a8cb).

In [ ]:
# Download dependencies for pos_tag()
nltk.download('averaged_perceptron_tagger')

In [ ]:
## define schema for returned result from the udf function
## the returned result is a list of tuples
schema = ArrayType(StructType([
            StructField('f1', StringType()),
            StructField('f2', StringType())    ]))

sent_to_tag_words_udf = udf(lambda x: nltk.pos_tag(x), schema)

In [ ]:
# Transform data
d6 = d5.withColumn("tagged_words", sent_to_tag_words_udf("words"))
d6.show(5) 

##### **Frequency Distribution Plot**

It gives us information about the number of times a word has occurred within a sentence.

In [ ]:
from nltk.probability import FreqDist

row = d6.select('words').toPandas().iloc[0,0]
fd = FreqDist(row) 
fd.plot(30, cumulative= False)
plt.show()

From the above plot it can be seen that in the first row, the word 'Man' has occurred twice.

##### **Chunking**
Chunking is the process of grouping similar words together based on the nature of the word. It is the process of segmenting and labeling multitokens. Let's see how to do a noun phrase chunking on the tagged words data frame from the previous step.

First we need to define a udf function that chunks noun phrases from a list of pos-tagged words.

In [ ]:
# define a udf function to chunk noun phrases from pos-tagged words
grammar = "NP: {<DT>?<JJ>*<NN>}"
chunk_parser = nltk.RegexpParser(grammar)
chunk_parser_udf = udf(lambda x: str(chunk_parser.parse(x)), StringType())

In [ ]:
# Transform data
d7 = d6.withColumn("NP_chunk", chunk_parser_udf("tagged_words"))

In [ ]:
d7.select('NP_chunk').show(1, truncate= False)

#### Load data

**Use Parquet file to store data**

In [ ]:
d7.write.format("parquet").mode("overwrite").save("transformed_text_parquet_data")

**Read data from Parquet file**

In [ ]:
df_text_parquet = spark.read.format("parquet").load("transformed_text_parquet_data")

In [ ]:
df_text_parquet.show(5)

**Store the data as a `json file`**

In [ ]:
d7.write.format("json").mode("overwrite").save('transformed_text_json_data.json')

**Read data from `json` to spark dataframe**

In [ ]:
df_text_json = spark.read.format("json").load('transformed_text_json_data.json')

In [ ]:
df_text_json.show(5)